In [5]:
from bs4 import BeautifulSoup
import urllib2
import pandas as pd

In [37]:
def get_locations(soup):
    locations = []
    base_link = 'https://www.privateproperty.co.za'
    for a in soup.findChildren("a"):
        try:
            if a['class'][0] == 'listingResultV2':
                l = urllib2.urlopen(base_link+a['href'])
                reader = BeautifulSoup(l, 'html.parser')
                desc = reader.find('div', attrs={'id':"descriptionParagraph"}).get_text().strip()
                locations.append([a['href'].split('/')[-2],desc])
        except KeyError:
            continue
    return locations

In [38]:
def clean_listings(listings, final_listings, vacant_land):
    for j in range(len(listings)):
        current_listing = listings[j]
        attr= []
        for div in current_listing.findAll('div'):
            feature = div.get_text().strip()
            if feature !='':
                attr.append(feature)
        clean_listing = {}
        try:
            clean_listing['title']= attr[0]
            clean_listing['price'] = attr[1]
            clean_listing['bedrooms'] = attr[3]
            clean_listing['bathrooms'] = attr[4]
            clean_listing['garage'] = attr[5]
            clean_listing['location'] = locations[j][0]
            clean_listing['description'] = locations[j][1]
            clean_listing['other'] = attr[5:]
            final_listings.append(clean_listing)
        except IndexError:
            vacant_land.append(attr)
    return final_listings, vacant_land

In [39]:
all_listings = []
vacant_land = []

In [43]:
areas = ['pretoria-central-and-old-east/31','northern-pretoria/29','pretoria-far-east/994','pretoria-east/991']
for area in areas:
    base_page = 'https://www.privateproperty.co.za/for-sale/gauteng/pretoria/'+area+'?page='
    for i in range(1,55):
        quote_page = base_page+str(i)
        page = urllib2.urlopen(quote_page)
        soup = BeautifulSoup(page, 'html.parser')
        locations = get_locations(soup)
        listings = soup.findAll('div', attrs={'class':'infoHolder'})
        final_listings = clean_listings(listings, all_listings, vacant_land)[0]
        vacant = clean_listings(listings, all_listings, vacant_land)[1]

In [44]:
df = pd.DataFrame(final_listings)

In [46]:
df.shape

(8850, 8)

In [47]:
df.to_csv('listings2.csv', sep= ',', encoding = 'utf-8')